In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Imports

import os
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [ ]:
# Função personalizada para ajustar a saturação sem perder a cor original
def adjust_saturation(image, saturation_factor):
    image_pil = transforms.functional.adjust_saturation(image, saturation_factor)
    return image_pil

In [ ]:
# Função personalizada para ajustar o contraste sem perder a cor original
def adjust_contrast(image, contrast_factor):
    image_pil = transforms.functional.adjust_contrast(image, contrast_factor)
    return image_pil

In [ ]:
# Função personalizada para ajustar o brilho sem perder a cor original
def adjust_brightness(image, brightness_factor):
    image_pil = transforms.functional.adjust_brightness(image, brightness_factor)
    return image_pil

Carregar dados


In [ ]:
# Classe personalizada para lidar com a classe ImageFolder e adicionar o caminho ao retorno
class ImageFolderWithPath(ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target, path

In [ ]:
# Transformação de pré-processamento
data_transform = transforms.Compose([
    transforms.Lambda(lambda x: adjust_contrast(x, contrast_factor=1)),
    transforms.Lambda(lambda x: adjust_brightness(x, brightness_factor=1)),
    transforms.Lambda(lambda x: adjust_saturation(x, saturation_factor=1)),
])

In [ ]:
main_dir = os.path.join("gdrive", "MyDrive", "IA901-2023S1", "data")

# Diretório dos dados
data_dir = os.path.join(main_dir, "interim", "filtered")

# Criar diretório para salvar as imagens
save_dir_root = os.path.join(main_dir, "interim", "normalized") 
os.makedirs(save_dir_root, exist_ok=True)

In [ ]:
# Percorrer os conjuntos de dados (treino, validação e teste)
for dataset_name in ['TRAIN', 'VAL', 'TEST']:
    # Criar diretório para o conjunto de dados atual
    save_dir = os.path.join(save_dir_root, dataset_name)
    os.makedirs(save_dir, exist_ok=True)

    # Carregar o conjunto de dados com a classe personalizada ImageFolderWithPath
    dataset = ImageFolderWithPath(os.path.join(data_dir, dataset_name), transform=data_transform)

    # Percorrer as imagens do conjunto de dados
    for i, (img, target, path) in enumerate(dataset):
        # Obter o diretório de origem da imagem e a classe da imagem
        source_dir = os.path.dirname(path)
        class_dir = dataset.classes[target]

        # Obter o caminho relativo da imagem em relação ao diretório de origem
        relative_path = os.path.relpath(path, start=source_dir)

        # Criar o diretório de destino para a classe e salvar a imagem aumentada
        save_class_dir = os.path.join(save_dir, class_dir)
        os.makedirs(save_class_dir, exist_ok=True)

        save_path = os.path.join(save_class_dir, relative_path)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        # Salvar a imagem em formato RGB usando o Pillow (PIL)
        img.save(save_path, format='JPEG')

In [ ]:
#0.936782   Saturação em 1 contrast em 0 e brilho em 0
#0.9253     Saturação em 1.1
#0.908046   Saturação em 0.8
#0.942529 contrast  em 1.1 Best val Acc: 0.942529
#0.919540   contrast  em 1.2
#0.931034   Brilho em 1.1
#0.925287   Brilho em 0.9